RetrieverQueryEngine

In [ ]:
# 관련 모듈 및 클래스 임포트
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceSplitter

# 문서 로드 및 인덱스 생성
documents = SimpleDirectoryReader(input_files=["../data/txt_example.txt"]).load_data()

# 2. 노드 파서 설정 수정
node_parser = SentenceSplitter(
    chunk_size=100,        # 청크 크기 증가
    chunk_overlap=20,        # 오버랩 제거
    separator="\n"        # 빈 줄을 기준으로 분할
)

# 문서 분할
nodes = node_parser.get_nodes_from_documents(documents)

# 총 노드 수 확인
print(f"총 노드 수: {len(nodes)}")


In [ ]:
# 벡터 인덱스 생성
index = VectorStoreIndex(nodes)

In [ ]:
# 리트리버 생성 (벡터 유사도 검색)
retriever = index.as_retriever(similarity_top_k=5)

# 리트리버 테스트
retrieved_nodes = retriever.retrieve("인공지능에 대해서 요약해줘")
print(f"\n검색된 노드 수: {len(retrieved_nodes)}\n")

for i, node in enumerate(retrieved_nodes, 1):
    print(f"\n--- 노드 {i} ---")
    print(node.text)


In [ ]:
# 후처리기 생성 (유사도 0.7 이상만 필터링)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.8)

# 응답 생성기 생성 (트리 요약 모드, 상세 로깅)
response_synthesizer = get_response_synthesizer(
    response_mode="compact",
    verbose=True
)

# 쿼리 엔진 생성
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[postprocessor]
)

# 쿼리 실행
response = query_engine.query("인공지능에 대해서 요약해줘")
print(response)
